In [7]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns




# Acceso a la Data Recolectada y su Transformacion

In [26]:
# Primero Accesamos a la data recolectada
with open('DataRecolectada - 1.json') as f:
    data_original = json.load(f)

In [27]:
# Generamos los nombres de las columnas del DF
columnas = ['Nombre','Sexo','Edo_Laboral','Edo_Civil','Edad','Diag_Previo','Tristeza','Pesimismo','Fracaso','Perdida_de_Placer',\
'Sentimientos_de_Culpa','Sentimientos_de_Castigo','Disconformidad_con_uno_mismo','Autocritica',\
'Pensamientos_o_Deseos_Suicidas','Llanto','Agitación','Perdida_de_Interés','Indecisión','Desvalorización','Perdida_de_Energía',\
'Cambios_en_los_Hábitos_de_Sueño','Irritabilidad','Cambios_en_el_Apetito','Dificultad_de_Concentración',\
'Cansancio_o_Fatiga','Perdida_de_Interés_en_el_Sexo', 'Respuesta_Recibida']

In [28]:
# Generamos el DF
df = pd.DataFrame.from_dict(data_original,orient='index',columns=columnas)
df.head()

,Nombre,Sexo,Edo_Laboral,Edo_Civil,Edad,Diag_Previo,Tristeza,Pesimismo,Fracaso,Perdida_de_Placer,...,Indecisión,Desvalorización,Perdida_de_Energía,Cambios_en_los_Hábitos_de_Sueño,Irritabilidad,Cambios_en_el_Apetito,Dificultad_de_Concentración,Cansancio_o_Fatiga,Perdida_de_Interés_en_el_Sexo,Respuesta_Recibida
1531026146,Ricardo,Hombre,Desempleado no Buscando,Casado,28,NO,D,D,D,D,...,D,D,C,G,D,G,D,D,D,Khhd
601887628,Reynaldo,Hombre,Empleado,Union Libre,25,SI,A,A,A,A,...,A,A,A,A,A,A,A,A,A,Ooi
1437527766,Carlos,Hombre,Empleado,Soltero,25,NO,A,A,A,B,...,A,A,B,C,B,G,B,A,B,Normal
33942341,Guillermo Arturo,Hombre,Empleado,Union Libre,35,NO,A,A,A,B,...,A,A,A,A,A,A,A,A,A,Animo
1772055088,Asairi,Hombre,Empleado,Casado,36,NO,A,A,C,A,...,A,A,A,A,A,A,A,A,A,"Ánimo, tú puedes"


In [81]:
# Como debemos de obtener las calificaciones por cada usuario, convertimos las respuestas obtenidas a numeros
# las sumamos y obtenemos la calificacion final.

def Valores_Respuestas_Usuario(respuestas_usuario):    
    valor_respuestas_numerico = []
    
    f = open('CuestionarioBeck.json','r')
    content = f.read()
    cuestionario = json.loads(content)

    for i in range(len(respuestas_usuario)):
        resp_user = respuestas_usuario[i]
        tema = cuestionario['Temas'][i]['Tema']
        data = cuestionario['Temas'][i]['Data']
        for x in range(len(data)):
            resp = data[x]['Inciso']
            valor = data[x]['Valor']
            if resp_user == resp:
                valor_respuestas_numerico.append(valor)
    
    return sum(valor_respuestas_numerico)

In [94]:
# Generamos un nuevo diccionario con la data transformada
data_valor = dict()

for i in data:
    try: 
        respuestas = data[i][6:-1]
        valores = Valores_Respuestas_Usuario(respuestas)
        data_valor.update({i:valores})
    except:
        data_valor.update({i:'null'})

In [96]:
# Lo convertimos en un DF
data_valor = pd.DataFrame.from_dict(data_valor,orient='index')
data_valor.head()

,0
1531026146,59
601887628,0
1437527766,12
33942341,1
1772055088,3


In [98]:
data_valor.shape

(47, 1)

In [99]:
df.shape

(47, 28)

In [111]:
# En ambos DF tenemos el mismo numero de registros, por lo que podemos unirlos. 
data = pd.concat([df,data_valor],axis=1)
data.drop('Nombre',axis=1,inplace=True)
data.rename(columns={0:'Calificacion_Final'},inplace=True)

In [114]:
# Obtenemos la data final, sin nombres y ya con la calificacion final de cada usuario
data.head()

,Sexo,Edo_Laboral,Edo_Civil,Edad,Diag_Previo,Tristeza,Pesimismo,Fracaso,Perdida_de_Placer,Sentimientos_de_Culpa,...,Desvalorización,Perdida_de_Energía,Cambios_en_los_Hábitos_de_Sueño,Irritabilidad,Cambios_en_el_Apetito,Dificultad_de_Concentración,Cansancio_o_Fatiga,Perdida_de_Interés_en_el_Sexo,Respuesta_Recibida,Calificacion_Final
1531026146,Hombre,Desempleado no Buscando,Casado,28,NO,D,D,D,D,D,...,D,C,G,D,G,D,D,D,Khhd,59
601887628,Hombre,Empleado,Union Libre,25,SI,A,A,A,A,A,...,A,A,A,A,A,A,A,A,Ooi,0
1437527766,Hombre,Empleado,Soltero,25,NO,A,A,A,B,A,...,A,B,C,B,G,B,A,B,Normal,12
33942341,Hombre,Empleado,Union Libre,35,NO,A,A,A,B,A,...,A,A,A,A,A,A,A,A,Animo,1
1772055088,Hombre,Empleado,Casado,36,NO,A,A,C,A,A,...,A,A,A,A,A,A,A,A,"Ánimo, tú puedes",3


# Analisis de la Data

In [122]:
# Vamos a inciar analizando las respuestas recibidas por los usuarios cuando comentan a familiares o amigos que se
# sienten tristes o tienen depresion (o sespecha de)

respuestas_recibidas = data.Respuesta_Recibida

# Primero eliminamos los nulos
respuestas_recibidas = respuestas_recibidas.dropna()


45